In [1]:
import urllib3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO
import json
from headers import *
import controllers

import importlib
importlib.reload(controllers)
controller = controllers.FuturesDataController()

In [2]:
controller.update_twse_data("2024/09/09")

爬取日期: 20240909
成功連接到 MongoDB!
MongoDB 連接已關閉


UnboundLocalError: cannot access local variable 'success' where it is not associated with a value

In [ ]:
df = controller.crawler_twse_total_return("2024/09/09")
df

In [ ]:
df2 = controller.crawler_twse_data("2024/09/09")
df2

In [2]:
url = "https://www.twse.com.tw/fund/TWT38U"
date = "20240909"
params = {
    'date':f'{date}'
}

response = requests.get(url,data=params,headers=TWSE_DOWN)

In [ ]:
data = json.loads(response.text)
data

In [ ]:
columns = data['fields']
rows = data['data']
df = pd.DataFrame(rows,columns=columns).iloc[:,:6]
df

In [ ]:
df['鉅額成交'] = 0
df.loc[df.iloc[:,0] == '*','鉅額成交'] = 1 
df

In [ ]:
df = df.replace(',','',regex=True)
df['date'] = date
df['date'] = pd.to_datetime(df['date'])
df = df[['date'] + [col for col in df.columns if col!='date']]
df

In [ ]:
df = controller.crawler_twse_trade_amount("2024/09/09")
df

In [ ]:
url = "https://www.twse.com.tw/rwd/fund/BFI82U"
date = "20240909"
params = {
    'response':'json',
    'dayDate':f'{date}',
    'type' : 'day'

}
response = requests.get(url,data=params,headers=TWSE_DOWN)
response.raise_for_status()

response.text

In [5]:
data = json.loads(response.text)

columns = data['fields']
rows = data['data']

df = pd.DataFrame(rows,columns = columns)

In [ ]:
df.iloc[:,1:] = df.iloc[:,1:].apply(lambda x:x.str.replace(',','').astype('int64'),axis=1)
df

In [ ]:
df['date'] = "2024/09/09"
df['date'] = pd.to_datetime(df['date'])
df

In [ ]:
df =df[df['單位名稱'] != "合計"]
df

In [ ]:
df = df[['date'] + [col for col in df.columns if col != 'date']]
df